# Training a ViT based classifier for 20 classes of ImageNet dataset

## Setup: Imports and Device Configuration

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import shutil
from tqdm import tqdm
import time
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Configuration and Hyperparameters

In [ ]:
FULL_IMAGENET_PATH = '/path/to/your/imagenet' 
SUBSET_PATH = './ImageNet20'
NUM_CLASSES = 20
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CHANNELS = 3
D_MODEL = 384
NUM_HEADS = 6
NUM_LAYERS = 6
MLP_RATIO = 4
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.05

## Dataset Preparation: Creating the ImageNet Subset

In [ ]:
import os
from datasets import load_dataset, DatasetDict

IMAGENET_20_SYNSETS = [
    'n02113186','n02099601','n02123045','n02124075','n02871525','n03085013',
    'n03126707','n03417042','n03445777','n03770679','n03888257','n03930630',
    'n04141975','n04209133','n04254680','n01855672','n01514859','n02410509',
    'n02422699','n02480495'
]

SYNSET_TO_HUMAN_LABEL = {
    'n01514859': 'cock',
    'n01855672': 'goose',
    'n02099601': 'Eskimo dog, husky',
    'n02113186': 'Cardigan, Cardigan Welsh corgi',
    'n02123045': 'tabby, tabby cat',
    'n02124075': 'Egyptian cat',
    'n02410509': 'bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis',
    'n02422699': 'impala, Aepyceros melampus',
    'n02480495': 'gorilla, Gorilla gorilla',
    'n02871525': 'bookshop, bookstore, bookstall',
    'n03085013': 'computer keyboard, keypad',
    'n03126707': 'crane',
    'n03417042': 'garbage truck, dustcart',
    'n03445777': 'golf ball',
    'n03770679': 'minibus',
    'n03888257': 'parachute, chute',
    'n03930630': 'pizza, pizza pie',
    'n04141975': 'safe',
    'n04209133': 'snowplow, snowplough',
    'n04254680': 'sports car, sport car'
}

OUT = "./ImageNet20_hf"

if not os.path.exists(OUT):
    print("Preparing dataset for the first time...")
    ds_id = "benjamin-paine/imagenet-1k-256x256"
    train_full = load_dataset(ds_id, split="train")
    val_full   = load_dataset(ds_id, split="validation")
    all_class_names = train_full.features["label"].names
    name_to_id = {name: i for i, name in enumerate(all_class_names)}
    TARGET_CLASS_NAMES = [SYNSET_TO_HUMAN_LABEL[s] for s in IMAGENET_20_SYNSETS]
    missing = [name for name in TARGET_CLASS_NAMES if name not in name_to_id]
    if missing:
        raise RuntimeError(f"Could not find the following class names in the dataset: {missing}")

    tgt_ids = {name_to_id[name] for name in TARGET_CLASS_NAMES}
    
    print("Filtering for 20 classes...")
    train_20 = train_full.filter(lambda ex: ex["label"] in tgt_ids, num_proc=4)
    val_20   = val_full.filter(lambda ex: ex["label"] in tgt_ids, num_proc=4)

    print("Remapping labels to 0-19 range...")
    sorted_target_names = [SYNSET_TO_HUMAN_LABEL[s] for s in sorted(IMAGENET_20_SYNSETS)]
    remap = {name_to_id[name]: i for i, name in enumerate(sorted_target_names)}
    
    train_final = train_20.map(lambda ex: {"label": remap[ex["label"]]}, num_proc=4)
    val_20_remapped = val_20.map(lambda ex: {"label": remap[ex["label"]]}, num_proc=4)

    print("Splitting validation set into validation and test sets...")
    val_test_split = val_20_remapped.train_test_split(test_size=0.5, seed=42, stratify_by_column="label")
    
    final_dataset = DatasetDict({
        "train": train_final, 
        "val": val_test_split['train'], 
        "test": val_test_split['test']
    })

    final_dataset.save_to_disk(OUT)
    print(f"--- Dataset saved to {OUT} ---")
else:
    print(f"Dataset already exists at {OUT}. Skipping preparation.")

Dataset already exists at ./ImageNet20_hf. Skipping preparation.


## Data Loading: Transforms and DataLoaders

In [ ]:
from datasets import load_from_disk
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

final_dataset = load_from_disk(OUT)
train_dataset_hf = final_dataset['train']
val_dataset_hf = final_dataset['val']
test_dataset_hf = final_dataset['test']

def apply_train_transforms(examples):
    examples['pixel_values'] = [train_transform(image.convert("RGB")) for image in examples['image']]
    return examples

def apply_val_test_transforms(examples):
    examples['pixel_values'] = [val_test_transform(image.convert("RGB")) for image in examples['image']]
    return examples

train_dataset_hf.set_transform(apply_train_transforms)
val_dataset_hf.set_transform(apply_val_test_transforms)
test_dataset_hf.set_transform(apply_val_test_transforms)

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

train_loader = DataLoader(train_dataset_hf, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset_hf, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset_hf, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, collate_fn=collate_fn)

print("\n--- DataLoaders Ready ---")
print(f"Training samples:   {len(train_dataset_hf)}")
print(f"Validation samples: {len(val_dataset_hf)}")
print(f"Test samples:       {len(test_dataset_hf)}")


--- DataLoaders Ready ---
Training samples:   25729
Validation samples: 500
Test samples:       500


## Vision Transformer (ViT) Model Implementation

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, d_model):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels, d_model, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        
        self.qkv = nn.Linear(d_model, d_model * 3)
        self.proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.dropout(x)
        return x

class MLP(nn.Module):
    def __init__(self, d_model, mlp_ratio, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, int(d_model * mlp_ratio))
        self.act = nn.GELU()
        self.fc2 = nn.Linear(int(d_model * mlp_ratio), d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, mlp_ratio, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.norm2 = nn.LayerNorm(d_model)
        self.mlp = MLP(d_model, mlp_ratio, dropout)
    
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, n_classes, d_model, n_heads, n_layers, mlp_ratio):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, d_model)
        num_patches = (img_size // patch_size) ** 2

        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, d_model))
        
        self.encoder = nn.Sequential(*[
            TransformerEncoder(d_model, n_heads, mlp_ratio) for _ in range(n_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, n_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        
        x = self.encoder(x)
        x = self.norm(x)
        
        cls_token_final = x[:, 0]
        x = self.head(cls_token_final)
        
        return x

## Training Setup: Model, Optimizer, Loss, and Scaler

In [8]:
model = VisionTransformer(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=NUM_CHANNELS,
    n_classes=NUM_CLASSES,
    d_model=D_MODEL,
    n_heads=NUM_HEADS,
    n_layers=NUM_LAYERS,
    mlp_ratio=MLP_RATIO
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params / 1e6:.2f}M")

Total trainable parameters: 11.03M


## Defining the Training and Validation Loops

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0.0
    
    loop = tqdm(loader, desc="Training")
    for batch in loop:
        images = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(images)
            loss = criterion(outputs, labels)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        loop.set_postfix(loss=loss.item())

    return running_loss / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        loop = tqdm(loader, desc="Validating")
        for batch in loop:
            images = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(images)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    return val_loss / len(loader.dataset), accuracy

## Running the Full Training Process

In [17]:
best_val_acc = 0.0
history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

print("Starting training...")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    epoch_duration = time.time() - epoch_start_time
    
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Val Acc: {val_acc:.2f}% | "
          f"Time: {epoch_duration:.2f}s")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'vit_best_model.pth')
        print(f"New best model saved with accuracy: {best_val_acc:.2f}%")

total_training_time = time.time() - start_time
print(f"\nTraining finished in {total_training_time/60:.2f} minutes.")
print(f"Best validation accuracy: {best_val_acc:.2f}%")

Starting training...


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.09it/s]


Epoch 1/50 | Train Loss: 5.1865 | Val Loss: 4.9396 | Val Acc: 5.40% | Time: 142.76s
New best model saved with accuracy: 5.40%


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.54it/s]


Epoch 2/50 | Train Loss: 4.6815 | Val Loss: 4.4494 | Val Acc: 5.60% | Time: 144.78s
New best model saved with accuracy: 5.60%


...

Epoch 48/50 | Train Loss: 0.9353 | Val Loss: 1.1107 | Val Acc: 64.80% | Time: 146.92s


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.45it/s]


Epoch 49/50 | Train Loss: 0.9329 | Val Loss: 1.1288 | Val Acc: 65.80% | Time: 146.89s


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.24it/s]

Epoch 50/50 | Train Loss: 0.9075 | Val Loss: 1.1404 | Val Acc: 65.20% | Time: 146.66s

Training finished in 122.23 minutes.
Best validation accuracy: 66.00%


## Experiment with Different Number of heads

In [ ]:
HEADS_TO_TEST = [4, 8]
experiment_results = {}
for n_heads in HEADS_TO_TEST:
    print(f"\n{'='*50}")
    print(f"  STARTING EXPERIMENT: {n_heads} ATTENTION HEADS")
    print(f"{'='*50}\n")
    if D_MODEL % n_heads != 0:
        print(f"Skipping {n_heads} heads: D_MODEL ({D_MODEL}) is not divisible by {n_heads}.")
        continue

    model = VisionTransformer(
        img_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        in_channels=NUM_CHANNELS,
        n_classes=NUM_CLASSES,
        d_model=D_MODEL,
        n_heads=n_heads,
        n_layers=NUM_LAYERS,
        mlp_ratio=MLP_RATIO
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model with {n_heads} heads has {total_params / 1e6:.2f}M trainable parameters.")

    best_val_acc = 0.0
    model_save_path = f'vit_heads_{n_heads}_best.pth'
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

    print(f"Starting training for {n_heads}-head model...")
    start_time = time.time()

    for epoch in range(EPOCHS):
        epoch_start_time = time.time()
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        history[f'train_loss_{n_heads}'] = train_loss
        history[f'val_loss_{n_heads}'] = val_loss
        history[f'val_acc_{n_heads}'] = val_acc
        
        epoch_duration = time.time() - epoch_start_time
        
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Time: {epoch_duration:.2f}s")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"--> New best model saved to {model_save_path} with accuracy: {best_val_acc:.2f}%")

    total_training_time = time.time() - start_time
    print(f"\nTraining for {n_heads}-head model finished in {total_training_time/60:.2f} minutes.")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"\n--- Evaluating best {n_heads}-head model on the TEST set ---")
    final_model = VisionTransformer(img_size=IMAGE_SIZE, patch_size=PATCH_SIZE, in_channels=NUM_CHANNELS,
                                    n_classes=NUM_CLASSES, d_model=D_MODEL, n_heads=n_heads,
                                    n_layers=NUM_LAYERS, mlp_ratio=MLP_RATIO).to(device)
    final_model.load_state_dict(torch.load(model_save_path))
    
    test_loss, test_acc = validate(final_model, test_loader, criterion, device)
    print(f"Final Test Accuracy for {n_heads} heads: {test_acc:.2f}%")
    experiment_results[n_heads] = {
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'training_time_min': total_training_time / 60
    }
print(f"\n\n{'='*50}")
print(f"  EXPERIMENT SUMMARY: EFFECT OF NUMBER OF HEADS")
print(f"{'='*50}")
print(f"{'Heads':<10} | {'Best Val Acc (%)':<20} | {'Final Test Acc (%)':<20} | {'Train Time (min)':<20}")
print(f"-"*75)
for n_heads, results in experiment_results.items():
    print(f"{n_heads:<10} | {results['best_val_acc']:<20.2f} | {results['test_acc']:<20.2f} | {results['training_time_min']:<20.2f}")


  STARTING EXPERIMENT: 4 ATTENTION HEADS

Model with 4 heads has 11.03M trainable parameters.
Starting training for 4-head model...


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.68it/s]


Epoch 1/50 | Train Loss: 2.4379 | Val Loss: 2.0361 | Val Acc: 37.20% | Time: 135.44s
--> New best model saved to vit_heads_4_best.pth with accuracy: 37.20%


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.71it/s]


Epoch 2/50 | Train Loss: 2.0386 | Val Loss: 1.8754 | Val Acc: 41.00% | Time: 134.55s
--> New best model saved to vit_heads_4_best.pth with accuracy: 41.00%


...

Epoch 49/50 | Train Loss: 0.7535 | Val Loss: 1.1263 | Val Acc: 65.60% | Time: 148.85s


Validating: 100%|██████████| 8/8 [00:01<00:00,  5.97it/s]


Epoch 50/50 | Train Loss: 0.7239 | Val Loss: 1.0895 | Val Acc: 66.80% | Time: 148.44s

Training for 8-head model finished in 123.49 minutes.
Best validation accuracy: 70.60%

--- Evaluating best 8-head model on the TEST set ---


Validating: 100%|██████████| 8/8 [00:01<00:00,  5.92it/s]

Final Test Accuracy for 8 heads: 67.40%


  EXPERIMENT SUMMARY: EFFECT OF NUMBER OF HEADS
Heads      | Best Val Acc (%)     | Final Test Acc (%)   | Train Time (min)    
---------------------------------------------------------------------------
4          | 66.80                | 67.60                | 113.64              
8          | 70.60                | 67.40                | 123.49              


## ViT Architecture

In [ ]:
import torch
import torch.nn as nn
import math

class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, d_model):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels, d_model, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.qkv = nn.Linear(d_model, d_model * 3)
        self.proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.scale = self.head_dim ** -0.5

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.n_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.dropout(x)
        return x

class MLP(nn.Module):
    def __init__(self, d_model, mlp_ratio, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, int(d_model * mlp_ratio))
        self.act = nn.GELU()
        self.fc2 = nn.Linear(int(d_model * mlp_ratio), d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, mlp_ratio, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.norm2 = nn.LayerNorm(d_model)
        self.mlp = MLP(d_model, mlp_ratio, dropout)
    
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, n_classes, d_model, n_heads, n_layers, mlp_ratio, pos_embed_type='learnable'):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, d_model)
        num_patches = (img_size // patch_size) ** 2
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        
        if pos_embed_type == 'learnable':
            print("Using Learnable Positional Embedding")
            self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, d_model))
        elif pos_embed_type == 'sine':
            print("Using Sinusoidal Positional Embedding")
            pe = torch.zeros(num_patches + 1, d_model)
            position = torch.arange(0, num_patches + 1, dtype=torch.float).unsqueeze(1)
            div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)
            pe = pe.unsqueeze(0)
            self.register_buffer('pos_embed', pe)
        else:
            print("Not using any Positional Embedding")
            self.pos_embed = None

        self.encoder = nn.Sequential(*[
            TransformerEncoder(d_model, n_heads, mlp_ratio) for _ in range(n_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, n_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        if self.pos_embed is not None:
            x = x + self.pos_embed
        
        x = self.encoder(x)
        x = self.norm(x)
        
        cls_token_final = x[:, 0]
        x = self.head(cls_token_final)
        
        return x

## ViT Training

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

POS_EMBEDS_TO_TEST = ['learnable', 'sine', None]
FIXED_NUM_HEADS = 4
experiment_results = {}

for pos_embed_type in POS_EMBEDS_TO_TEST:
    pos_embed_name = str(pos_embed_type)
    
    print(f"\n{'='*60}")
    print(f"  STARTING EXPERIMENT: {pos_embed_name.upper()} POSITIONAL EMBEDDING")
    print(f"{'='*60}\n")
    
    model = VisionTransformer(
        img_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        in_channels=NUM_CHANNELS,
        n_classes=NUM_CLASSES,
        d_model=D_MODEL,
        n_heads=FIXED_NUM_HEADS,
        n_layers=NUM_LAYERS,
        mlp_ratio=MLP_RATIO,
        pos_embed_type=pos_embed_type
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model has {total_params / 1e6:.2f}M trainable parameters.")

    best_val_acc = 0.0
    model_save_path = f'vit_pos_{pos_embed_name.lower()}_best.pth'
    history = {}

    print(f"Starting training for {pos_embed_name} model...")
    start_time = time.time()

    for epoch in range(EPOCHS):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_save_path)
            print(f"--> New best model saved to {model_save_path} with accuracy: {best_val_acc:.2f}%")

    total_training_time = time.time() - start_time
    print(f"\nTraining for {pos_embed_name} model finished in {total_training_time/60:.2f} minutes.")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")

    print(f"\n--- Evaluating best {pos_embed_name} model on the TEST set ---")
    final_model = VisionTransformer(img_size=IMAGE_SIZE, patch_size=PATCH_SIZE, in_channels=NUM_CHANNELS,
                                    n_classes=NUM_CLASSES, d_model=D_MODEL, n_heads=FIXED_NUM_HEADS,
                                    n_layers=NUM_LAYERS, mlp_ratio=MLP_RATIO, pos_embed_type=pos_embed_type).to(device)
    final_model.load_state_dict(torch.load(model_save_path))
    
    test_loss, test_acc = validate(final_model, test_loader, criterion, device)
    print(f"Final Test Accuracy for {pos_embed_name} model: {test_acc:.2f}%")

    experiment_results[pos_embed_name] = {
        'best_val_acc': best_val_acc,
        'test_acc': test_acc,
        'training_time_min': total_training_time / 60
    }

print(f"\n\n{'='*75}")
print(f"  EXPERIMENT SUMMARY: EFFECT OF POSITIONAL EMBEDDING (Heads={FIXED_NUM_HEADS})")
print(f"{'='*75}")
print(f"{'Positional Embedding':<25} | {'Best Val Acc (%)':<20} | {'Final Test Acc (%)':<20} | {'Train Time (min)':<20}")
print(f"-"*90)
for pos_embed_name, results in experiment_results.items():
    print(f"{pos_embed_name:<25} | {results['best_val_acc']:<20.2f} | {results['test_acc']:<20.2f} | {results['training_time_min']:<20.2f}")


  STARTING EXPERIMENT: LEARNABLE POSITIONAL EMBEDDING

Using Learnable Positional Embedding
Model has 11.03M trainable parameters.
Starting training for learnable model...


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.48it/s]


Epoch 1/50 | Train Loss: 2.4275 | Val Loss: 2.1593 | Val Acc: 33.20%
--> New best model saved to vit_pos_learnable_best.pth with accuracy: 33.20%


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.52it/s]


Epoch 2/50 | Train Loss: 2.0644 | Val Loss: 1.9522 | Val Acc: 39.60%
--> New best model saved to vit_pos_learnable_best.pth with accuracy: 39.60%


...

Epoch 49/50 | Train Loss: 0.7344 | Val Loss: 1.1049 | Val Acc: 68.00%
--> New best model saved to vit_pos_none_best.pth with accuracy: 68.00%


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.41it/s]


Epoch 50/50 | Train Loss: 0.7187 | Val Loss: 1.0915 | Val Acc: 67.60%

Training for None model finished in 111.66 minutes.
Best validation accuracy: 68.00%

--- Evaluating best None model on the TEST set ---
Not using any Positional Embedding


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.29it/s]

Final Test Accuracy for None model: 69.40%


  EXPERIMENT SUMMARY: EFFECT OF POSITIONAL EMBEDDING (Heads=4)
Positional Embedding      | Best Val Acc (%)     | Final Test Acc (%)   | Train Time (min)    
------------------------------------------------------------------------------------------
learnable                 | 70.40                | 68.80                | 113.00              
sine                      | 70.40                | 70.20                | 112.26              
None                      | 68.00                | 69.40                | 111.66              


## FCNN

In [10]:
class FCFNNClassifier(nn.Module):
    def __init__(self, img_size=224, in_channels=3, num_classes=20):
        super(FCFNNClassifier, self).__init__()
        input_features = in_channels * img_size * img_size
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_features, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

print("\n--- Starting FCFNN Experiment ---")
fcfnn_model = FCFNNClassifier(img_size=IMAGE_SIZE, in_channels=NUM_CHANNELS, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(fcfnn_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

total_params = sum(p.numel() for p in fcfnn_model.parameters() if p.requires_grad)
print(f"FCFNN Model - Total trainable parameters: {total_params / 1e6:.2f}M")

best_val_acc = 0.0
fcfnn_history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

print("Starting FCFNN training...")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    
    train_loss = train_one_epoch(fcfnn_model, train_loader, criterion, optimizer, scaler, device)
    val_loss, val_acc = validate(fcfnn_model, val_loader, criterion, device)
    
    fcfnn_history['train_loss'].append(train_loss)
    fcfnn_history['val_loss'].append(val_loss)
    fcfnn_history['val_acc'].append(val_acc)
    
    epoch_duration = time.time() - epoch_start_time
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Time: {epoch_duration:.2f}s")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(fcfnn_model.state_dict(), 'fcfnn_best_model.pth')
        print(f"New best FCFNN model saved with accuracy: {best_val_acc:.2f}%")

total_training_time = time.time() - start_time
print(f"\nFCFNN training finished in {total_training_time/60:.2f} minutes.")
print(f"FCFNN best validation accuracy: {best_val_acc:.2f}%")

print("\n--- Evaluating best FCFNN model on the final test set ---")
final_fcfnn_model = FCFNNClassifier(img_size=IMAGE_SIZE, in_channels=NUM_CHANNELS, num_classes=NUM_CLASSES).to(device)
final_fcfnn_model.load_state_dict(torch.load('fcfnn_best_model.pth'))
fcfnn_test_loss, fcfnn_test_acc = validate(final_fcfnn_model, test_loader, criterion, device)
print(f"\nFinal FCFNN Test Accuracy: {fcfnn_test_acc:.2f}%")
print(f"Final FCFNN Test Loss: {fcfnn_test_loss:.4f}")


--- Starting FCFNN Experiment ---
FCFNN Model - Total trainable parameters: 154.68M
Starting FCFNN training...


Validating: 100%|██████████| 8/8 [00:00<00:00, 12.33it/s]


Epoch 1/50 | Train Loss: 4.5035 | Val Loss: 2.9220 | Val Acc: 10.20% | Time: 45.35s
New best FCFNN model saved with accuracy: 10.20%


Validating: 100%|██████████| 8/8 [00:00<00:00, 12.00it/s]


Epoch 2/50 | Train Loss: 2.9744 | Val Loss: 2.8780 | Val Acc: 11.60% | Time: 45.64s
New best FCFNN model saved with accuracy: 11.60%


...

Epoch 49/50 | Train Loss: 2.8580 | Val Loss: 2.7460 | Val Acc: 14.00% | Time: 45.53s


Validating: 100%|██████████| 8/8 [00:00<00:00, 12.09it/s]


Epoch 50/50 | Train Loss: 2.8554 | Val Loss: 2.7313 | Val Acc: 13.00% | Time: 45.57s

FCFNN training finished in 38.82 minutes.
FCFNN best validation accuracy: 20.80%

--- Evaluating best FCFNN model on the final test set ---


Validating: 100%|██████████| 8/8 [00:00<00:00, 11.57it/s]


Final FCFNN Test Accuracy: 21.40%
Final FCFNN Test Loss: 2.6615


## CNN

In [11]:
class CNNClassifier(nn.Module):
    def __init__(self, in_channels=3, num_classes=20):
        super(CNNClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

print("\n--- Starting CNN Experiment ---")
cnn_model = CNNClassifier(in_channels=NUM_CHANNELS, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(cnn_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

total_params = sum(p.numel() for p in cnn_model.parameters() if p.requires_grad)
print(f"CNN Model - Total trainable parameters: {total_params / 1e6:.2f}M")

best_val_acc = 0.0
cnn_history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

print("Starting CNN training...")
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    
    train_loss = train_one_epoch(cnn_model, train_loader, criterion, optimizer, scaler, device)
    val_loss, val_acc = validate(cnn_model, val_loader, criterion, device)
    
    cnn_history['train_loss'].append(train_loss)
    cnn_history['val_loss'].append(val_loss)
    cnn_history['val_acc'].append(val_acc)
    
    epoch_duration = time.time() - epoch_start_time
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Time: {epoch_duration:.2f}s")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(cnn_model.state_dict(), 'cnn_best_model.pth')
        print(f"New best CNN model saved with accuracy: {best_val_acc:.2f}%")

total_training_time = time.time() - start_time
print(f"\nCNN training finished in {total_training_time/60:.2f} minutes.")
print(f"CNN best validation accuracy: {best_val_acc:.2f}%")

print("\n--- Evaluating best CNN model on the final test set ---")
final_cnn_model = CNNClassifier(in_channels=NUM_CHANNELS, num_classes=NUM_CLASSES).to(device)
final_cnn_model.load_state_dict(torch.load('cnn_best_model.pth'))
cnn_test_loss, cnn_test_acc = validate(final_cnn_model, test_loader, criterion, device)
print(f"\nFinal CNN Test Accuracy: {cnn_test_acc:.2f}%")
print(f"Final CNN Test Loss: {cnn_test_loss:.4f}")


--- Starting CNN Experiment ---
CNN Model - Total trainable parameters: 27.79M
Starting CNN training...


Validating: 100%|██████████| 8/8 [00:00<00:00,  8.15it/s]


Epoch 1/50 | Train Loss: 2.5994 | Val Loss: 2.0940 | Val Acc: 33.20% | Time: 75.25s
New best CNN model saved with accuracy: 33.20%


Validating: 100%|██████████| 8/8 [00:01<00:00,  7.73it/s]


Epoch 2/50 | Train Loss: 2.1537 | Val Loss: 1.9239 | Val Acc: 38.80% | Time: 75.25s
New best CNN model saved with accuracy: 38.80%


...

Epoch 49/50 | Train Loss: 0.6011 | Val Loss: 0.5790 | Val Acc: 83.20% | Time: 75.53s


Validating: 100%|██████████| 8/8 [00:00<00:00,  8.17it/s]


Epoch 50/50 | Train Loss: 0.5695 | Val Loss: 0.4994 | Val Acc: 85.20% | Time: 75.60s
New best CNN model saved with accuracy: 85.20%

CNN training finished in 63.28 minutes.
CNN best validation accuracy: 85.20%

--- Evaluating best CNN model on the final test set ---


Validating: 100%|██████████| 8/8 [00:01<00:00,  7.68it/s]


Final CNN Test Accuracy: 84.00%
Final CNN Test Loss: 0.5392
